In [6]:
import tensorflow as tf

In [7]:
import os

In [10]:
_URL = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'

zip_file = tf.keras.utils.get_file(origin=_URL, fname='flower_photos.tgz',extract=True,cache_subdir='/content')

base_dir = os.path.join(os.path.dirname(zip_file),'flower_photos')

228813984/228813984 [==============================] - 1s 0us/step


In [11]:
IMAGE_SIZE = 224
BATCH_SIZE = 64
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, # to make file size smaller
    validation_split=0.2 # test or nueral-network
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
)

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [13]:
print(train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt','w') as f:
  f.write(labels)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [14]:
IMG_SHAPE = (IMAGE_SIZE,IMAGE_SIZE,3) # it's RGB that's why

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
    )

9406464/9406464 [==============================] - 0s 0us/step


In [15]:
base_model.trainable = False

In [16]:
model = tf.keras.Sequential([
     base_model,
    tf.keras.layers.Conv2D(32,3,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(5,activation='softmax')
])

In [19]:
from keras.api._v2.keras import metrics
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [20]:
epochs = 10

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

Epoch 1/10
46/46 [==============================] - 172s 4s/step - loss: 0.7425 - accuracy: 0.7452 - val_loss: 0.4458 - val_accuracy: 0.8276
Epoch 2/10
46/46 [==============================] - 166s 4s/step - loss: 0.2851 - accuracy: 0.8989 - val_loss: 0.3962 - val_accuracy: 0.8482
Epoch 3/10
46/46 [==============================] - 166s 4s/step - loss: 0.1968 - accuracy: 0.9275 - val_loss: 0.4305 - val_accuracy: 0.8659
Epoch 4/10
46/46 [==============================] - 166s 4s/step - loss: 0.1145 - accuracy: 0.9571 - val_loss: 0.4438 - val_accuracy: 0.8741
Epoch 5/10
46/46 [==============================] - 154s 3s/step - loss: 0.0756 - accuracy: 0.9775 - val_loss: 0.4868 - val_accuracy: 0.8728
Epoch 6/10
46/46 [==============================] - 165s 4s/step - loss: 0.0560 - accuracy: 0.9837 - val_loss: 0.4672 - val_accuracy: 0.8741
Epoch 7/10
46/46 [==============================] - 164s 4s/step - loss: 0.0364 - accuracy: 0.9918 - val_loss: 0.4895 - val_accuracy: 0.8632
Epoch 8/10
46

In [21]:
saved_model_dir = ''
tf.saved_model.save(model,saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert() # tflite is used for on device ML

with open('model.tflite','wb') as f:
  f.write(tflite_model)

In [22]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>